In [23]:
from nltk.corpus import movie_reviews

In [24]:
movie_reviews

<CategorizedPlaintextCorpusReader in '/Users/aayushkhurana/nltk_data/corpora/movie_reviews'>

In [25]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Error loading movie_reviews: <urlopen error [Errno 8]
[nltk_data]     nodename nor servname provided, or not known>


False

In [26]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aayushkhurana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aayushkhurana/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
movie_reviews.categories()

['neg', 'pos']

In [4]:
#This dataset contains many files which are divided into two categoriesd : Negative and Positive movie reviews
#To load all the files we can use fileids
movie_reviews.fileids()
len(movie_reviews.fileids())

2000

In [5]:
#So this dataset contains 2000 files of which 1000 are positive reviews files and 1000 are negative reviews file
#To get only negative review files we can pass the category 
movie_reviews.fileids('neg')
len(movie_reviews.fileids('neg'))


1000

In [6]:
#To get all the words of any file we can use 
movie_reviews.words(movie_reviews.fileids()[5])
#Here we have listed all the words in the fifth file 


['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]

In [7]:
#Code to make a list of tuples of the type : [('Words of file 1','category of file 1')]
documents=[]
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid),category))
documents[:5]

[(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'),
 (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'),
 (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'),
 (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'),
 (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')]

In [8]:
#So we have 1000 negative tuples and 1000 positive tuples
#So let us shuffle them using random.shuffle
import random
random.shuffle(documents)
documents[:5]

[(['it', "'", 's', 'now', 'the', 'anniversary', 'of', ...], 'neg'),
 (['these', 'days', 'the', 'lack', 'of', 'originality', ...], 'neg'),
 (['paul', 'verhoeven', ',', 'the', 'dutch', 'auteur', ...], 'neg'),
 (['bruce', 'willis', 'stars', 'as', 'malcolm', ',', ...], 'pos'),
 (['shakespeare', 'in', 'love', 'is', 'quite', ...], 'neg')]

In [21]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

In [10]:
lemmatizer=WordNetLemmatizer()


In [11]:
w="aayush"
pos=pos_tag([w])
pos[0][1]

'NN'

In [12]:
#function to return the part of speech required by the lemmatizer to lemmatize
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [13]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])        #pos_tag expects an array , hence we have passed [w] and not just w 
            clean_word = lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))   #pos array's zeroth row and its first column is retrieved
            output_words.append(clean_word.lower())
    return output_words

In [14]:
#Creating the stopwords list
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
len(stops)

179

In [58]:
#To remove punctuations , we add the list of punctuations to stop words itself
import string
punctuation = set(string.punctuation)
print(len(punctuation))
stops.update(punctuation)
len(stops)

32


211

In [16]:
document_clean=[]
for i in range(len(documents)):
    document_clean.append((clean_review(documents[i][0]),documents[i][1]))

In [17]:
len(document_clean)

2000

In [18]:
train_documents=document_clean[0:1500]
test_documents=document_clean[1500:]


In [19]:
all_words=[]
for doc in train_documents:
    all_words = all_words + doc[0]       #When two arrays are added the secomd array gets appended to the first

In [26]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)       #To get the 3000 top frequency words 
features = [i[0] for i in common ]     #To store only the top 3000 words without their frequencies

In [27]:
#Now we need to create a dictionary to see if the top frequency word comes in that document or not and return true or false
def get_feature_dict(words):
    current_features={}
    words_set=set(words)   #Just creates a set of words that the function receives as the array-words
    for w in features:
        current_features[w] = w in words_set
    return current_features
    

In [28]:
#Training data tuples of the type ((dictionary stating true or false,category)) for each document 
training_data=[]
for doc in train_documents:
    training_data.append((get_feature_dict(doc[0]),doc[1]))


In [29]:
#Testing data tuples of the type ((dictionary stating true or false,category)) for each document 
testing_data=[]
for doc in test_documents:
    testing_data.append((get_feature_dict(doc[0]),doc[1]))



In [30]:
#We created our data in this format as sklearn nltk classifier requires data to be in this format only .
#What's the format : Array of tuples in which each tuple has a dictionary (with key as the feature and value as true or false)
#and the category that document belongs to
from nltk import NaiveBayesClassifier


In [31]:
#Syntax would be a little different than that we used to follow
classifier = NaiveBayesClassifier.train(training_data)


In [32]:
#Now in nltk we have a accuracy function which requires 2 arguments : our classifier and the testing data
nltk.classify.accuracy(classifier,testing_data)

0.792

In [33]:
classifier.show_most_informative_features(15)

Most Informative Features
             outstanding = True              pos : neg    =     10.6 : 1.0
                    zeta = True              neg : pos    =      9.2 : 1.0
                  alicia = True              neg : pos    =      9.2 : 1.0
               ludicrous = True              neg : pos    =      8.6 : 1.0
                   jolie = True              neg : pos    =      8.5 : 1.0
                   damon = True              pos : neg    =      8.0 : 1.0
                lifeless = True              neg : pos    =      8.0 : 1.0
                   bland = True              neg : pos    =      7.7 : 1.0
              uninspired = True              neg : pos    =      7.4 : 1.0
                   mulan = True              pos : neg    =      6.9 : 1.0
                  seagal = True              neg : pos    =      6.7 : 1.0
                 idiotic = True              neg : pos    =      6.5 : 1.0
                     sat = True              neg : pos    =      6.5 : 1.0

In [34]:
#Now we will use sklkearn classifiers to train our model
#using SVC 
from sklearn .svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier  

In [35]:
svc = SVC()
classifier_sklearn = SklearnClassifier(svc)    #The Sklearnclassifier at the back uses svc object 


In [36]:
classifier_sklearn.train(training_data)

/Users/aayushkhurana/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


<SklearnClassifier(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))>

In [37]:
nltk.classify.accuracy(classifier_sklearn,testing_data)

0.79

In [38]:
#Now lets use random forest classifier 
from sklearn.ensemble import RandomForestClassifier

In [39]:
rfc = RandomForestClassifier()

In [40]:
classifier_sklearn1 = SklearnClassifier(rfc)    #The Sklearnclassifier at the back uses rfc object 

In [41]:
classifier_sklearn1.train(training_data)

/Users/aayushkhurana/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


<SklearnClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))>

In [42]:
nltk.classify.accuracy(classifier_sklearn1,testing_data)

0.662

In [43]:
#Now to use the sklearn classifier directly (without the help of nltk) we need to connvert the data into the format :
#X is a 2D array of values in which each column is a feature and rows are the frequencies of that feature in a given document
#and Y is a 1D array of target values(categories)

In [44]:
#So lets convert our nltk dataset into sklearn required format of X and Y using CountVectorizer
#First let us apply this for some dummy data
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
train_set={"the sky is blue","the sun is bright"}
count_vec=CountVectorizer(max_features=3)
a = count_vec.fit_transform(train_set)       #Here 'a' is a sparse matrix 
a.todense()

matrix([[1, 1, 1],
        [0, 1, 1]], dtype=int64)

In [46]:
#To see the values of features for the sparse matrix we can use
count_vec.get_feature_names()

['blue', 'is', 'the']

In [47]:
" ".join(train_set)

'the sky is blue the sun is bright'

In [61]:
#This example shown above clearly explains that our CountVectorizer requires data in the form of sentences. Hence we join our documents dataset like :
text_documents= [" ".join(doc) for doc,category in document_clean]
len(text_documents)

2000

In [62]:
categories = [cat for doc,cat in document_clean]      #Y has to be 1D dimensinal target values

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
x_train,x_test,y_train,y_test=train_test_split(text_documents,categories,random_state=1)

In [65]:
count_vec1 = CountVectorizer(max_features=2000,ngram_range=(1,3),max_df=0.8,min_df=0.05)   #ngram_range specifies combination of words (Here 1 and 2 combinations are allowed)

In [66]:
x_train_features = count_vec1.fit_transform(x_train)

In [70]:
x_train_features.todense(),x_train_features.shape,len(x_train)

(matrix([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 1, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 1, ..., 1, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int64), (1500, 1047), 1500)

In [253]:
#count_vec1.get_feature_names()

In [254]:
x_test_features = count_vec1.transform(x_test)

In [255]:
x_test_features

<500x1046 sparse matrix of type '<class 'numpy.int64'>'
	with 67336 stored elements in Compressed Sparse Row format>

In [256]:
svc1=SVC()

In [257]:
svc1.fit(x_train_features,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [258]:
svc1.score(x_test_features,y_test)

0.81

In [249]:
#N-grams are basically the combination of words. combination of 2 words are called bigrams , 3 are called trigrams and so on
#For eg: 'Not good' is a bigram and 'Not very Good' is a trigram
#N-gram is a property of CountVectorizer and is specified as count_vec1 = CountVectorizer(max_features=2000,ngram_range=(2,3))
#This means only bigrams and trigrams will be our features 

#TF/IDF stands for Term Frequency and Inverse Document Frequency. IDF = 1/TF
#For a word , it's document frequency is defined as the no of documents that contain that word. The point here is that if a word 
#is present in a lot of documents,it's not that important. We like to work with those words that have low document frequency.
#Ideally,a word that comes in few documents can help in classification way better than a word that comes in all the documents.
#To implement this we have two more parameters in CountVectorizer : max_df and min_df 
#max_df = 0.8 means if a word is present in 80% of the documents , I am not interested in that word.
#min_df = 0.1 means if a word is not present in 10% of the documents , I am not interested in that word.
#max_df kind of helps us remove stopwords from our vocabulary
#One interesting thing is that if a word like e-mail is present in a lot of documents and is not a stopword , then it's document frequency will be very high.




